In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 5
learning_rate = 0.01

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='/home/hasu/Documents/github/content/MNIST/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='/home/hasu/Documents/github/content/MNIST/',
                                          train=False, 
                                          transform=transforms.ToTensor())


In [8]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [9]:
len(train_loader)

600

In [10]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [11]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [12]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 0.0347
Epoch [1/10], Step [200/600], Loss: 0.0505
Epoch [1/10], Step [300/600], Loss: 0.0355
Epoch [1/10], Step [400/600], Loss: 0.0293
Epoch [1/10], Step [500/600], Loss: 0.0620
Epoch [1/10], Step [600/600], Loss: 0.0084
Epoch [2/10], Step [100/600], Loss: 0.1426
Epoch [2/10], Step [200/600], Loss: 0.0205
Epoch [2/10], Step [300/600], Loss: 0.0828
Epoch [2/10], Step [400/600], Loss: 0.0341
Epoch [2/10], Step [500/600], Loss: 0.0470
Epoch [2/10], Step [600/600], Loss: 0.0839
Epoch [3/10], Step [100/600], Loss: 0.0403
Epoch [3/10], Step [200/600], Loss: 0.0158
Epoch [3/10], Step [300/600], Loss: 0.0620
Epoch [3/10], Step [400/600], Loss: 0.0589
Epoch [3/10], Step [500/600], Loss: 0.0248
Epoch [3/10], Step [600/600], Loss: 0.1668
Epoch [4/10], Step [100/600], Loss: 0.0619
Epoch [4/10], Step [200/600], Loss: 0.0543
Epoch [4/10], Step [300/600], Loss: 0.0430
Epoch [4/10], Step [400/600], Loss: 0.0694
Epoch [4/10], Step [500/600], Loss: 0.0095
Epoch [4/10

In [17]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 92.57 %
